In [1]:
import scipy
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

from src.IterativeFitting import IterativeFitting as IF
from src.CorrFuncs import covariance_matrix, trend_est

In the Smith example, $p$ and $z$ are calculated from the initial frequencies of cases and controls. So maybe just use the crude estimates for $p$ and $z$.

Do we construct estimates for vx and for Lx everytime? I don't think so if we look at step 3. It seems like we input them from the information you give the algorithm and information that we get from study reports.

In [2]:
x = np.array([2,6,11])
Nx = np.array([337,167,186,212])
M1x = 451
Lx = np.array([np.log(0.80),np.log(1.16),np.log(1.57)])
# vx = np.array([0.0542,0.0563,0.0563])
# vx = np.array([0.03,0.06,0.09]) # Case that breaks!!!!!!!!
vx = np.array([.09,.01,.001])
# a0 = 165
a0 = 10
A0x = np.array([74,90,122])
# b0 = 172
b0 = 10
B0x = np.array([93,96,90])
B0x_sum = np.sum(B0x) + b0
# vx = 1/a0 + 1/b0 + 1/A0x + 1/B0x

Defining initial parameters of $p$ and $z$.

In [3]:
p0 = b0 / B0x_sum
z0 = B0x_sum / M1x

In [4]:
p0 = 0.38
z0 = 1.0

In [5]:
p0

0.38

In [6]:
z0

1.0

Defining a function that allows us to construct confidence intervals

Get sum of squared differences and then check to see how microsoft solver works... Also look at R code probably, it will be easier. This is just an optimization problem for the following function. We need to find b0 and a0 that optimize the function. Maybe cvxpy?

According to the R code, we want to use scipy minimize. a0 and b0 will be the parameters we optimize over. Then the function should be longer but contain something like sum_sq_diff. 

In [7]:
def f_LogLik(C_Val):
    a0x = C_Val[0]
    b0x = C_Val[1]

    Vextra = vx - 1/a0x - 1/b0x
    Est_A = (1 + (a0x/b0x)*np.exp(Lx))/(Vextra)
    Est_B = (1 + b0x/(a0x*np.exp(Lx)))/(Vextra)
    
    SumA = a0x + np.sum(Est_A)
    SumB = b0x + np.sum(Est_B)

    p1 = b0x / SumB
    F1 = ((p0-p1)/p0)**2
    z1 = SumB/SumA
    F2 = ((z0-z1)/z0)**2

    return F1 + F2

In [8]:
a0_b0_res = scipy.optimize.minimize(f_LogLik,[a0,b0])

In [9]:
a0_b0_res

      fun: 1.073028475844847e-14
 hess_inv: array([[1013.87830293, -153.0751213 ],
       [-153.0751213 ,   29.84287803]])
      jac: array([-7.91182781e-09, -5.49970476e-08])
  message: 'Optimization terminated successfully.'
     nfev: 78
      nit: 14
     njev: 26
   status: 0
  success: True
        x: array([45.67998079, 21.02355672])

In [10]:
a0, b0 = a0_b0_res.x[0], a0_b0_res.x[1]

In [11]:
a0

45.67998079327342

In [12]:
b0

21.023556721478023

In [13]:
a0/b0

2.1727998453566126

In [14]:
Vextra = vx - 1/a0 - 1/b0
A = (1 + (a0/b0)*np.exp(Lx))/(Vextra)
B = (1 + b0/(a0*np.exp(Lx)))/(Vextra)

In [15]:
A

array([133.2938563 , -59.20986252, -64.43881725])

In [16]:
B

array([ 76.68323465, -23.49180254, -18.88983595])

In [17]:
vx - 1/a0 - 1/b0

array([ 0.02054288, -0.05945712, -0.06845712])

In [18]:
Lx

array([-0.22314355,  0.14842001,  0.45107562])

In [19]:
s = np.sqrt(1/A + 1/B + 1/a0 + 1/b0)
r = ((1/np.outer(s,s))*(1/a0 + 1/b0))[np.triu_indices(3,k=1)]
c = r*np.sqrt((np.outer(vx,vx))[np.triu_indices(3,k=1)])
C1 = np.diag(vx)
triu_indices = np.triu_indices(3,k=1)
tril_indices = np.tril_indices(3,k=-1)
C2 = C1.copy()
C2[triu_indices] = c
C = C2.copy()
C[tril_indices] = c

In [20]:
C

array([[0.09      , 0.06945712, 0.06945712],
       [0.06945712, 0.01      , 0.06945712],
       [0.06945712, 0.06945712, 0.001     ]])

In [21]:
Cinv = np.linalg.inv(C)
vb_star = 1/(np.dot(x,np.dot(Cinv,x)))
b_star = vb_star*(np.dot(x,np.dot(Cinv,Lx)))

In [22]:
b_star

0.07018390139021692

In [23]:
vb_star

-0.0003299683936761447